# Pydantic Basics

## Introduction

Pydantic provides data validation using Python type hints:
- **Automatic validation** - Validates data on creation
- **Type coercion** - Converts compatible types automatically
- **Clear error messages** - Shows exactly what's wrong
- **JSON serialization** - Easy conversion to/from JSON
- **IDE support** - Full autocomplete and type checking

**Perfect for AI:** Validate API responses, config files, and user inputs!

## Learning Objectives

1. Create Pydantic models with BaseModel
2. Add field validation and constraints
3. Handle nested models
4. Parse and validate JSON
5. Use Pydantic with APIs

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional
import json

## 1. Basic Pydantic Models

In [ ]:
class User(BaseModel):
    """Simple user model."""
    name: str
    age: int
    email: str

# Create instance - validates automatically!
user = User(name="Alice", age=30, email="alice@example.com")
print(user)
print(f"Name: {user.name}, Age: {user.age}")

# Try invalid data
try:
    bad_user = User(name="Bob", age="thirty", email="bob@example.com")
except ValidationError as e:
    print(f"\nValidation Error:\n{e}")

## 2. Field Validation and Constraints

In [ ]:
from pydantic import Field, field_validator

class Product(BaseModel):
    """Product with field constraints."""
    name: str = Field(min_length=1, max_length=100)
    price: float = Field(gt=0, description="Price must be positive")
    quantity: int = Field(ge=0, description="Quantity cannot be negative")
    tags: List[str] = Field(default_factory=list)
    
    @field_validator('name')
    @classmethod
    def name_must_not_be_empty(cls, v: str) -> str:
        if not v.strip():
            raise ValueError('Name cannot be empty or whitespace')
        return v.strip()

# Valid product
product = Product(name="Laptop", price=999.99, quantity=10, tags=["electronics"])
print(product)

# Invalid: negative price
try:
    bad_product = Product(name="Bad", price=-10, quantity=5)
except ValidationError as e:
    print(f"\nError: {e.errors()[0]['msg']}")

## 3. Optional Fields and Defaults

In [ ]:
class Config(BaseModel):
    """Configuration with optional fields."""
    api_key: str
    model: str = "gpt-3.5-turbo"
    max_tokens: int = 100
    temperature: Optional[float] = None
    timeout: float = 30.0

# Minimal config (uses defaults)
config1 = Config(api_key="sk-test")
print(f"Config 1: {config1}")

# Full config
config2 = Config(
    api_key="sk-test",
    model="gpt-4",
    max_tokens=500,
    temperature=0.7
)
print(f"\nConfig 2: {config2}")

## 4. Nested Models

In [ ]:
class Address(BaseModel):
    street: str
    city: str
    country: str

class Company(BaseModel):
    name: str
    address: Address
    employees: List[User]

# Create nested structure
company = Company(
    name="TechCorp",
    address=Address(
        street="123 Main St",
        city="San Francisco",
        country="USA"
    ),
    employees=[
        User(name="Alice", age=30, email="alice@tech.com"),
        User(name="Bob", age=25, email="bob@tech.com")
    ]
)

print(f"Company: {company.name}")
print(f"Location: {company.address.city}")
print(f"Employees: {len(company.employees)}")

## 5. JSON Parsing

In [ ]:
# From dict
user_data = {"name": "Charlie", "age": 35, "email": "charlie@example.com"}
user = User(**user_data)
print(f"From dict: {user}")

# From JSON string
json_str = '{"name": "David", "age": 40, "email": "david@example.com"}'
user = User.model_validate_json(json_str)
print(f"From JSON: {user}")

# To dict
user_dict = user.model_dump()
print(f"To dict: {user_dict}")

# To JSON
json_output = user.model_dump_json()
print(f"To JSON: {json_output}")

## 6. Type Coercion

In [ ]:
# Pydantic automatically converts compatible types
class Stats(BaseModel):
    count: int
    average: float
    is_valid: bool

# String numbers get converted!
stats = Stats(count="42", average="3.14", is_valid="true")
print(f"Count type: {type(stats.count)} = {stats.count}")
print(f"Average type: {type(stats.average)} = {stats.average}")
print(f"Is valid type: {type(stats.is_valid)} = {stats.is_valid}")

## Summary

### Key Features:
1. **BaseModel** - Base class for all models
2. **Field** - Add constraints and metadata
3. **Validators** - Custom validation logic
4. **Nested models** - Validate complex structures
5. **JSON support** - Parse and serialize easily
6. **Type coercion** - Automatic type conversion

### Next: Pydantic for AI Applications